# Announcements

### HW2 is due tonight by midnight (11:59pm Eastern).
  #### Remember to upload it as a .ipynb file
  #### Please reach out if you have any questions!

# Unsupervised learning of morphology is used to reduce data sparsity

We can consider how we can learn the best morphological "vocabulary" from data. For this, we can devise simple **unsupervised learning** algorithms to characterize regularities in our data that we can take advantage of. Without labels, we will try to find as many morphemes as possible so we can segment words into their component parts. (Figuring out what to call these morphemes, or identify what grammatical role they play, is a much harder question).

## Algorithms for morphology

There are many unsupervised learning algorithms for morphology. Nearly all of them make use of probability theory, such as **transition probabilities** and mutual information like we have computed before. One baseline algorithm for learning morphology-like sequences is the Morfessor algorithm, which we discuss below. We will also cover the Byte Pair Encoding (BPE) framework. Both of these are **bottom-up**, data-driven approaches to learning informative substrings that characterize word forms.

Note: There are elements of the Morfessor algorithm that are challenging until we get to later in the class. If you read the paper, be aware that some of the algorithms will be covered in class (EM or Expectation Maximization), and others will not (Viterbi).



# Morfessor algorithm (Creutz & Lagus, 2002) -- Assigned reading

Two major components:
* Lexicon (words or word-like units -- "constructions" -- and their properties)
* Grammar (a system that controls how these units combine into "compounds")

Two general assumptions:
* "Compounds" are formed by one or more "constructions"
  * Maximally, a word can have as many constructions as "atoms" (e.g., characters or letters)
  * Corollary: We cannot have "invisible" constructions (e.g., **zero derivation**)
* Each construction within a compound can occur **independently** -- words are effectively constructed at random

Original implementation (in Perl): http://morpho.aalto.fi/projects/morpho/morfessor1.0-utf8.perl


### Independence assumption

In terms of implementational details for parsing wordforms, what this means is that no aspect of parsing should be conditioned on the surrounding segmentations. This independence assumption is especially important for speed. It is much faster and easier to assess this probability:

<center>$\displaystyle \text{Cost(Source text)} = \sum_{\text{morph tokens}} - \text{log}(p(m_i))$</center>

than to assess the joint probability, which requires a _full co-occurrence table_. The joint probability:

<center>$\displaystyle \text{Cost(Source text)} = p(m_1, m_2, ..., m_k)$</center>

breaks down via the chain rule:

<center>$\displaystyle \text{Cost(Source text)} = p(m_k | m_1, m_2, ..., m_{k-1}) * p(m_1) ... p(m_k)$</center>

## Minimum Description Length
Morfessor works by two main prongs. First, the algorithm uses a recursive segmentation strategy to find what it calls "morphs" in the data. These "morphs" correspond to the "constructions" earlier. The algorithm tries to compute the cost of two aspects of the data:

* The cost of the system for segmentation (the cost of the "codebook", or the vocabulary of morphs; $\text{Cost}(\text{Codebook})$)
  * $\displaystyle \sum_{\text{types}} k * len(m_j)$
  * The length of a morph in characters = $len(m_j)$ -- Longer morphs have greater cost
  * Number of bits needed to encode a single character (constant) = $k$
* The cost of the corpus (how wrong the codebook is in characterizing the corpus; $\text{Cost}(\text{Source text})$)
  * $\displaystyle \sum_{\text{tokens}} - \text{log}(p(m_i))$

# Byte Pair Encoding (BPE)

Great discussion in Jurafsky and Martin (SP3)

Overall idea: Can we pretend morphology does not exist, and instead use a **greedy algorithm** to learn **subwords**?

## What is a subword?

A subword is a sequence of characters that is used to characterize a word instead of its actual spelling. Typically, subwords are learned from _statistical regularities_ in a corpus, specifically by assessing the frequencies of co-occurrences between sequences of characters in a corpus. 

Great introduction to subwords in NLP from Manning's CS224N lectures at Stanford: https://youtu.be/9oTHFx0Gg3Q?t=2489

## BPE learning algorithm: Compression algorithm

Starting with the smallest level of granularity (the individual character), BPE works by doing "merges" that are purely based on frequencies. 

The general idea is that you're looking for the most frequent sequence of two bytes that really ought to be combined. The more we can do this, the less redundant the different pieces are, and the more useful each piece becomes.

First, let's show how this might be applied to our tokenized abstracts.

In [1]:
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

file_path = '/content/drive/MyDrive/Teaching/Fall2021/' \
            'Computational Linguistics/Assignments/HW2/files'

abstracts = open(os.path.join(file_path, 'abstracts.tsv'), 'r').readlines()

Mounted at /content/drive


In [2]:
# neat code from https://leimao.github.io/blog/Byte-Pair-Encoding/
# REALLY great blog post ^^^^
import re, collections

def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens

vocab = get_vocab(os.path.join(file_path, 'abstracts.tsv'))

print('==========')
print('Tokens Before BPE')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('==========')

num_merges = 50
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens = get_tokens(vocab)
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')


Tokens Before BPE
Tokens: defaultdict(<class 'int'>, {'O': 18276, 'f': 406369, 'e': 2530614, 'n': 1551127, 's': 1430092, 'i': 1498014, 'v': 212797, '</w>': 3587048, 'l': 811914, 'a': 1680957, 'g': 426287, 'u': 582058, 'd': 719566, 't': 1803246, 'c': 731024, 'o': 1432167, '(': 34348, 'L': 30355, 'D': 17642, ')': 35579, 'h': 644761, 'r': 1297268, 'm': 518689, 'p': 522657, '.': 176819, 'R': 25110, 'w': 233330, 'k': 94075, 'b': 224354, 'H': 10099, ',': 161781, 'y': 225799, '-': 114801, 'T': 70785, '/': 5966, 'A': 33788, 'B': 16337, 'E': 37355, 'x': 89583, 'P': 20137, 'j': 12611, 'I': 33864, 'z': 20980, 'G': 12555, 'F': 16590, 'W': 40726, '1': 20678, '+': 864, '0': 22376, '9': 6970, '4': 5594, '2': 18704, 'q': 38951, 'M': 29649, 'C': 27013, '{': 68429, '`': 12768, '}': 68429, "'": 47768, ':': 7610, 'U': 9357, '7': 5913, '8': 6017, 'S': 35707, 'N': 28522, '\\': 45280, '_': 482, '3': 6721, '6': 5356, '5': 6389, 'K': 4387, ';': 3453, 'Y': 983, 'J': 2341, 'V': 5177, 'Z': 590, '%': 5826, 'X': 14

We can observe that each iteration of merges in the vocabulary does not guarantee many gains, especially at the beginning. But, already we have learned some endings and common pairs of letters, like `-ed`, `-ing`, `-al`, `-er`, `-tion`, etc. The first 30 words in our vocabulary contain the merges below:

In [3]:
list(vocab.keys())[0:30]

['O f f en s i v e</w>',
 'l an g u a g e</w>',
 'd e t e c tion </w>',
 '( O L D ) </w>',
 'h a s</w>',
 're c e i v ed</w>',
 'in c re as ing</w>',
 'at t en tion </w>',
 'd u e</w>',
 'to</w>',
 'i t s</w>',
 's o c i e t al</w>',
 'i m p a c t .</w>',
 'R e c en t</w>',
 'w or k </w>',
 's h o w s</w>',
 'th a t</w>',
 'b i d i re c tion al</w>',
 't r an s for m er</w>',
 'b as ed</w>',
 'm e th o d s</w>',
 'o b t a in</w>',
 'i m p re s s i v e</w>',
 'p er for m an c e</w>',
 'on </w>',
 'O L D .</w>',
 'H o w e v er ,</w>',
 's u ch </w>',
 'u s u al l y</w>',
 're l y</w>']

## BPE and Neural Networks

We can check out various BPE models available in the Huggingface group's python package known as [Transformers](https://huggingface.co/transformers/tokenizer_summary.html). 

BPE models are deployed widely throughout neural network models because neural networks are very large and data hungry and benefit from training optimization (dimensionality reduction) and sparsity hacks like this. We can compress a vocabulary of 

In [4]:
#install packages we need to get right tokenizers
!pip install transformers sentencepiece

In [5]:
from transformers import XLNetTokenizer, BertTokenizer

xl_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
b_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

print("Outputs of different wordpiece/subword tokenization models")
sent = 'Experimental results on benchmark datasets show that our approach'
tokenizers = {"XL": xl_tokenizer, "BERT": b_tokenizer}
for model in tokenizers:
  tokenizer = tokenizers[model]
  print(f"{model}: {tokenizer.tokenize(sent)}")

Outputs of different wordpiece/subword tokenization models
XL: ['▁Experimental', '▁results', '▁on', '▁benchmark', '▁dataset', 's', '▁show', '▁that', '▁our', '▁approach']
BERT: ['Experimental', 'results', 'on', 'bench', '##mark', 'data', '##sets', 'show', 'that', 'our', 'approach']


Use `json` builtin to read each line

In [6]:
from google.colab import files
import json

morphemes = files.upload()['morphemes.json'].decode('utf-8').split("\n")

Saving morphemes.json to morphemes (5).json


In [8]:
morphemes[-1]

'{"word": "holster", "roots": ["holster"], "prefixes": [], "suffixes": []}'

In [9]:
morphemes_list = []
for i, line in enumerate(morphemes):
  morphemes_list.append(json.loads(line))

# need to make this a bit nicer
morphemes_dict = {}
for i, morpheme_dict in enumerate(morphemes_list):
  word = morpheme_dict['word']
  morphemes_dict[word] = morpheme_dict

# Comparing BPE to gold segmentations

We are going to load in some gold segmentation data from a dataset called `morphemes.json` which is a string version of a `dictionary` on each line whose keys are `"roots"`, `"suffixes"` and `"prefixes"`. We can load these in using the `json` built-in python module, specifically `json.loads(line)`.

Aligning a BPE segmentation and a gold segmentation is pretty tricky. So, we'll do an easier thing. Basically, we are going to iterate over each word piece and check whether it is in the (1) roots, (2) suffixes, or (3) prefixes arrays within each dictionary for a word. 

Here, we might be curious whether BPE can recover what we know are real morphemes. We can take counts of times that BPE gave us a wordpiece that actually corresponded to a real morpheme for a given word. 

For that we need to do the following:

1. Look up the information about the word in a "gold" dataset (e.g., from English Lexicon Project)
2. Tokenize the word into word pieces
3. Iterate over each word piece to see:
  * Is it a root?
  * Is it a prefix?
  * Is it a suffix?
4. Count proportion of matches ("is-a") out of all word pieces we see

Ideally, a good BPE will capture linguistic intuitions. But, since it knows nothing about words per se, it might do very poorly in general. 

Question: Do you think any parts of the words will be easier for BPE to match?

In [11]:
counts = {'roots': [], 'suffixes': [], 'prefixes': []}
for word in morphemes_dict:
  word_morphology = morphemes_dict[word]
  segmented_word = b_tokenizer.tokenize(word)
  for wordpiece in segmented_word:
    # strip off special characters
    # turn hashtags into empty string
    wordpiece_clean = wordpiece.replace("##", "")
    # check roots
    in_roots = wordpiece_clean in word_morphology['roots']
    counts['roots'].append(in_roots)
    # check suffixes
    in_suffixes = wordpiece_clean in word_morphology['suffixes']
    counts['suffixes'].append(in_suffixes)
    # check prefixes
    in_prefixes = wordpiece_clean in word_morphology['prefixes']
    counts['prefixes'].append(in_prefixes)

In [17]:
for morpheme_type in counts:
  total_matches = sum(counts[morpheme_type]) # number of 1s in whole list
  total_pieces = len(counts[morpheme_type]) # number of wordpieces observed
  proportion_morphemes = total_matches / total_pieces
  print(f"Word pieces that are {morpheme_type} proportion: {proportion_morphemes}")

Word pieces that are roots proportion: 0.098331039229181
Word pieces that are suffixes proportion: 0.0973847212663455
Word pieces that are prefixes proportion: 0.1628527185134205


# Upcoming topics

Next week, we're going to be doing work with word meanings! The primary way that we will be looking at word meanings is using **matrices** and **vectors**. We will effectively be stepping up our nested dictionary approach to bigger numerical data structures.

If you missed this before, do take a look at the bag-of-words code from a couple of weeks ago. We're going to be scaling this up from individual document representations to whole corpora in order to approximate the meanings of words.

```python
def bag_of_words_vector(bag: dict, vocabulary: set):
  # create a vector -- approximate using a list
  bow_vector = []
  # loop through our fixed vocabulary
  # this will "fix" all the dimensions
  for word in vocabulary:
    word_frequency = bag[word]
    bow_vector.append(word_frequency)
  return bow_vector

# define dimensions for vectors as each word
words = []
for abstract in abstracts:
  words += word_tokenize(abstract)
vocabulary = set(words)

# build a vector for the very first abstract
# we expect to see a ton of 0s
bag_of_words_vector(bag=Counter(word_tokenize(abstracts[0])),
                    vocabulary=vocabulary)[0:100]

# in first abstract, what proportion of all words have non-zero values?
bow_first_abstract = bag_of_words_vector(
    bag=Counter(word_tokenize(abstracts[0])),
    vocabulary=vocabulary)

# convert our existing array into an array of only 0s and 1s
# this is a *binary valued* bag of words
nonzero = [x > 0 for x in bow_first_abstract]

number_nonzero_for_abstract = sum(nonzero)
vocabulary_size = len(vocabulary)

print(f"probability of non-zeros (proportion with values > 0):"
      f" {number_nonzero_for_abstract / vocabulary_size}")
```